# Modelo con GraphSage - Ensayo

In [1]:
# Instalar bibliotecas necesarias
!pip install networkx transformers pysentimiento pandas numpy scikit-learn

In [2]:
!pip uninstall torch torchvision torchaudio torch-sparse torch-scatter torch-geometric pyg-lib -y

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: torch_sparse 0.6.18+pt26cu124
Uninstalling torch_sparse-0.6.18+pt26cu124:
  Successfully uninstalled torch_sparse-0.6.18+pt26cu124
Found existing installation: torch_scatter 2.1.2+pt26cu124
Uninstalling torch_scatter-2.1.2+pt26cu124:
  Successfully uninstalled torch_scatter-2.1.2+pt26cu124
Found existing installation: torch_geometric 2.5.3
Uninstalling torch_geometric-2.5.3:
  Successfully uninstalled torch_geometric-2.5.3
Found existing installation: pyg-lib 0.4.0+pt26cu124
Uninstalling pyg-lib-0.4.0+pt26cu124:
  Successfully uninstalled pyg-lib-0.4.0+pt26

In [3]:
!pip install torch==2.6.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://download.pytorch.org/whl/cu124
  Using cached https://download.pytorch.org/whl/cu124/torch-2.6.0%2Bcu124-cp311-cp311-linux_x86_64.whl.metadata (28 kB)
  Using cached https://download.pytorch.org/whl/cu124/torchvision-0.21.0%2Bcu124-cp311-cp311-linux_x86_64.whl.metadata (6.1 kB)
  Using cached https://download.pytorch.org/whl/cu124/torchaudio-2.6.0%2Bcu124-cp311-cp311-linux_x86_64.whl.metadata (6.6 kB)
Using cached https://download.pytorch.org/whl/cu124/torch-2.6.0%2Bcu124-cp311-cp311-linux_x86_64.whl (768.5 MB)
Using cached https://download.pytorch.org/whl/cu124/torchvision-0.21.0%2Bcu124-cp311-cp311-linux_x86_64.whl (7.3 MB)
Using cached https://download.pytorch.org/whl/cu124/torchaudio-2.6.0%2Bcu124-cp311-cp311-linux_x86_64.whl (3.4 MB)


In [4]:
!pip install torch-scatter==2.1.2+pt26cu124 torch-sparse==0.6.18+pt26cu124 torch-geometric==2.5.3 -f https://data.pyg.org/whl/torch-2.6.0+cu124.html

Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
  Using cached https://data.pyg.org/whl/torch-2.6.0%2Bcu124/torch_scatter-2.1.2%2Bpt26cu124-cp311-cp311-linux_x86_64.whl (10.8 MB)
  Using cached https://data.pyg.org/whl/torch-2.6.0%2Bcu124/torch_sparse-0.6.18%2Bpt26cu124-cp311-cp311-linux_x86_64.whl (5.0 MB)
  Using cached torch_geometric-2.5.3-py3-none-any.whl.metadata (64 kB)
Using cached torch_geometric-2.5.3-py3-none-any.whl (1.1 MB)


In [5]:
!pip install pyg-lib==0.4.0+pt26cu124 -f https://data.pyg.org/whl/torch-2.6.0+cu124.html

Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
  Using cached https://data.pyg.org/whl/torch-2.6.0%2Bcu124/pyg_lib-0.4.0%2Bpt26cu124-cp311-cp311-linux_x86_64.whl (4.7 MB)


In [6]:
import torch
print(torch.__version__)  # Debería ser 2.6.0
print(torch.version.cuda)  # Debería ser 12.4 (para cu124)

import torch_geometric
print(torch_geometric.__version__)  # Debe imprimir la versión sin errores

2.6.0+cu124
12.4
2.5.3


In [7]:
print(torch.cuda.is_available())  # Debe ser True si hay GPU, sino False

False


In [8]:
# Importar bibliotecas
from google.colab import files
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from transformers import AutoTokenizer, AutoModel
import torch

In [ ]:
# Subir los archivos CSV
uploaded = files.upload()

In [9]:
# Cargar los CSV en DataFrames
emprendimientos = pd.read_csv('emprendimientos.csv')
publicaciones = pd.read_csv('publicaciones.csv')
comentarios = pd.read_csv('comentarios.csv')
seguidores = pd.read_csv('seguidores.csv')
emprendimiento_tematica = pd.read_csv('emprendimiento_tematica.csv')
municipios = pd.read_csv('municipios.csv')
alcances = pd.read_csv('alcance.csv')
tematicas = pd.read_csv('tematicas.csv')

In [10]:
# emprendimientos = pd.read_csv('emprendimientos.csv', encoding='utf-8-sig')

In [11]:
print("Seguidores:\n", seguidores.head())

Seguidores:
    id_seguidores  id_emprendimiento  cantidad
0              1                  1       312
1              2                  2       387
2              3                  3       173
3              4                  4       146
4              5                  5       231


In [12]:
print("Municipios:\n", municipios.head())

Municipios:
    id_municipio municipio
0             1   Popayán
1             2    Silvia
2             3    Totoró
3             4   Cajibío
4             5  Piendamó


In [13]:
print("Alcances:\n", alcances.head())

Alcances:
    id_alcance      tipo
0           1     Local
1           2  Regional


In [14]:
print("Temáticas:\n", tematicas.head())

Temáticas:
    id_tematica                  nombre
0            1  Agricultura sostenible
1            2               Reciclaje
2            3       Tecnología social
3            4              Artesanías
4            5       Salud comunitaria


In [15]:
print("Comentarios:\n", comentarios.head())

Comentarios:
    id_comentario                                         comentario  \
0              1  ¡Qué bacano! Voy este sábado a sembrar. 🌱 #Hue...   
1              2  ¿Dónde es el taller en Popayán, parce? #Huerto...   
2              3  Buen proyecto, pero falta más info del horario...   
3              4  Me encanta la idea, pero ojalá sea más largo. ...   
4              5  ¡Qué chimba! @JuanCa, vente al taller. 🙌 #Sost...   

   id_publicacion  
0               1  
1               1  
2               1  
3               1  
4               1  


## Construcción Grafo

In [16]:
# Inicializar el grafo
G = nx.Graph()

# Agregar nodos (emprendimientos)
for _, row in emprendimientos.iterrows():
    # Obtener temáticas asociadas
    temas = emprendimiento_tematica[emprendimiento_tematica['id_emprendimiento'] == row['id_emprendimiento']]['id_tematica'].tolist()
    # Atributos del nodo
    node_attrs = {
        'nombre_emprendimiento': row['nombre_emprendimiento'],
        'descripcion': row['descripcion'] if pd.notna(row['descripcion']) else '',
        'redes_sociales': row['redes_sociales'] if pd.notna(row['redes_sociales']) else '',
        'sitio_web': row['sitio_web'],
        'id_municipio_origen': row['id_municipio_origen'],
        'id_alcance': row['id_alcance'],
        'tematicas': temas
    }
    G.add_node(row['id_emprendimiento'], **node_attrs)

print(f"Nodos creados: {G.number_of_nodes()}")

Nodos creados: 60


In [17]:
# Crear aristas basadas en interacciones
for _, pub in publicaciones.iterrows():
    id_emprendimiento = pub['id_emprendimiento']
    # Conectar emprendimientos que comparten temáticas en publicaciones (simplificación, ya que no tenemos datos de quién comentó)
    related_emprendimientos = emprendimiento_tematica[
        emprendimiento_tematica['id_tematica'].isin(
            emprendimiento_tematica[emprendimiento_tematica['id_emprendimiento'] == id_emprendimiento]['id_tematica']
        )
    ]['id_emprendimiento'].unique()
    for rel_emp in related_emprendimientos:
        if rel_emp != id_emprendimiento and rel_emp in G.nodes:
            weight = pub['n_likes'] if pd.notna(pub['n_likes']) else 0
            G.add_edge(id_emprendimiento, rel_emp, weight=weight + 1)

# Crear aristas basadas en similitudes
for emp1 in G.nodes:
    for emp2 in G.nodes:
        if emp1 < emp2:  # Evitar duplicados
            emp1_attrs = G.nodes[emp1]
            emp2_attrs = G.nodes[emp2]
            # Similitud por temática
            temas_comunes = len(set(emp1_attrs['tematicas']) & set(emp2_attrs['tematicas']))
            # Similitud por municipio y alcance
            same_municipio = 1 if emp1_attrs['id_municipio_origen'] == emp2_attrs['id_municipio_origen'] else 0
            same_alcance = 1 if emp1_attrs['id_alcance'] == emp2_attrs['id_alcance'] else 0
            # Ponderar arista
            weight = temas_comunes + same_municipio + same_alcance
            if weight > 0:
                G.add_edge(emp1, emp2, weight=weight)

# Agregar peso basado en seguidores
for _, row in seguidores.iterrows():
    id_emprendimiento = row['id_emprendimiento']
    if id_emprendimiento in G.nodes:
        G.nodes[id_emprendimiento]['seguidores'] = row['cantidad']
        # Aumentar peso de aristas existentes según seguidores
        for emp2 in G.neighbors(id_emprendimiento):
            G[id_emprendimiento][emp2]['weight'] += row['cantidad'] / 1000  # Normalizar

print(f"Aristas creadas: {G.number_of_edges()}")

Aristas creadas: 1127


In [18]:
# Normalizar atributos numéricos
scaler = MinMaxScaler()

# Calcular total de likes por emprendimiento
likes_por_emprendimiento = publicaciones.groupby('id_emprendimiento')['n_likes'].sum().reset_index()
likes_por_emprendimiento.columns = ['id_emprendimiento', 'total_likes']

# Combinar likes y seguidores
emprendimientos_features = emprendimientos.merge(likes_por_emprendimiento, on='id_emprendimiento', how='left').fillna({'total_likes': 0})
emprendimientos_features = emprendimientos_features.merge(seguidores[['id_emprendimiento', 'cantidad']], on='id_emprendimiento', how='left').fillna({'cantidad': 0})

# Normalizar total_likes y cantidad
numeric_features = scaler.fit_transform(emprendimientos_features[['total_likes', 'cantidad']])
emprendimientos_features[['total_likes_norm', 'cantidad_norm']] = numeric_features

# Codificar atributos categóricos
onehot_encoder_municipio = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
municipio_encoded = onehot_encoder_municipio.fit_transform(emprendimientos[['id_municipio_origen']])
municipio_encoded_df = pd.DataFrame(municipio_encoded, columns=onehot_encoder_municipio.get_feature_names_out(['id_municipio_origen']))

onehot_encoder_alcance = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
alcance_encoded = onehot_encoder_alcance.fit_transform(emprendimientos[['id_alcance']])
alcance_encoded_df = pd.DataFrame(alcance_encoded, columns=onehot_encoder_alcance.get_feature_names_out(['id_alcance']))

# Codificar temáticas (matriz binaria)
tematica_encoded = np.zeros((len(emprendimientos), len(tematicas)))
for _, row in emprendimiento_tematica.iterrows():
    idx_emp = emprendimientos.index[emprendimientos['id_emprendimiento'] == row['id_emprendimiento']].tolist()[0]
    idx_tem = tematicas.index[tematicas['id_tematica'] == row['id_tematica']].tolist()[0]
    tematica_encoded[idx_emp, idx_tem] = 1
tematica_encoded_df = pd.DataFrame(tematica_encoded, columns=[f'tematica_{i}' for i in tematicas['id_tematica']])

# Combinar características numéricas y categóricas
features_df = pd.concat([emprendimientos_features[['id_emprendimiento', 'total_likes_norm', 'cantidad_norm']],
                         municipio_encoded_df, alcance_encoded_df, tematica_encoded_df], axis=1)

# Agregar características al grafo
for _, row in features_df.iterrows():
    if row['id_emprendimiento'] in G.nodes:
        G.nodes[row['id_emprendimiento']]['features'] = row.drop('id_emprendimiento').values

print("Dimensiones de las características:\n", features_df.shape)

Dimensiones de las características:
 (60, 39)


### Procesar Texto

In [19]:
# Cargar modelo BETO
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")
model = AutoModel.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
# Función para obtener embeddings de texto
def get_text_embedding(text):
    if not text or pd.isna(text):
        return np.zeros(768)  # Dimensión de BETO
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

#### Duración aproximada del procesamiento de textos: 20 MIN

In [ ]:
# SI ES LA PRIMERA VEZ QUE SE EJECUTA EL ENTORNO:

'''

import numpy as np
from google.colab import files

# Procesar textos
emprendimientos['descripcion_embedding'] = emprendimientos['descripcion'].apply(get_text_embedding)
publicaciones['contenido_embedding'] = publicaciones['contenido'].apply(get_text_embedding)
comentarios['comentario_embedding'] = comentarios['comentario'].apply(get_text_embedding)

# Guardar embeddings de descripción, contenido y comentario
np.save('descripcion_embeddings.npy', emprendimientos['descripcion_embedding'].to_numpy())
np.save('contenido_embeddings.npy', publicaciones['contenido_embedding'].to_numpy())
np.save('comentario_embeddings.npy', comentarios['comentario_embedding'].to_numpy())

# Descargar los archivos para guardarlos localmente
files.download('descripcion_embeddings.npy')
files.download('contenido_embeddings.npy')
files.download('comentario_embeddings.npy')

'''

In [ ]:
# Subir los archivos .npy si no están en el entorno
uploaded = files.upload()  # Sube descripcion_embeddings.npy, contenido_embeddings.npy, comentario_embeddings.npy

In [21]:
# Cargar embeddings desde los archivos .npy
descripcion_embeddings = np.load('descripcion_embeddings.npy', allow_pickle=True)
contenido_embeddings = np.load('contenido_embeddings.npy', allow_pickle=True)
comentario_embeddings = np.load('comentario_embeddings.npy', allow_pickle=True)

In [22]:
# Asignar embeddings a los DataFrames
emprendimientos['descripcion_embedding'] = list(descripcion_embeddings)
publicaciones['contenido_embedding'] = list(contenido_embeddings)
comentarios['comentario_embedding'] = list(comentario_embeddings)

# Verificar que los embeddings se cargaron correctamente
print("Ejemplo de embedding de descripción:", emprendimientos['descripcion_embedding'].iloc[0][:5])
print("Ejemplo de embedding de contenido:", publicaciones['contenido_embedding'].iloc[0][:5])
print("Ejemplo de embedding de comentario:", comentarios['comentario_embedding'].iloc[0][:5])

Ejemplo de embedding de descripción: [-0.58608776 -0.16883136 -0.05874122 -0.47675955  0.70658404]
Ejemplo de embedding de contenido: [-0.5049721  -1.00958     0.6562884  -0.7907445   0.41996792]
Ejemplo de embedding de comentario: [-0.46697903 -0.85226786  0.5786223  -0.2946373   0.12753987]


In [23]:
# Combinar embeddings por emprendimiento
for emp_id in G.nodes:
    # Embedding de la descripción
    desc_embedding = emprendimientos[emprendimientos['id_emprendimiento'] == emp_id]['descripcion_embedding'].iloc[0]

    # Promedio de embeddings de publicaciones
    pub_embeddings = publicaciones[publicaciones['id_emprendimiento'] == emp_id]['contenido_embedding']
    pub_embedding = np.mean([emb for emb in pub_embeddings if emb is not None], axis=0) if len(pub_embeddings) > 0 else np.zeros(768)

    # Promedio de embeddings de comentarios
    pub_ids = publicaciones[publicaciones['id_emprendimiento'] == emp_id]['id_publicacion'].tolist()
    com_embeddings = comentarios[comentarios['id_publicacion'].isin(pub_ids)]['comentario_embedding']
    com_embedding = np.mean([emb for emb in com_embeddings if emb is not None], axis=0) if len(com_embeddings) > 0 else np.zeros(768)

    # Combinar embeddings
    combined_embedding = (desc_embedding + pub_embedding + com_embedding) / 3
    G.nodes[emp_id]['text_features'] = combined_embedding

print("Ejemplo de embedding de texto:", G.nodes[emprendimientos['id_emprendimiento'].iloc[0]]['text_features'][:5])

Ejemplo de embedding de texto: [-0.39286777 -0.5032728   0.33132803 -0.5614875   0.5627695 ]


In [24]:
from torch_geometric.data import Data

# Combinar características
for emp_id in G.nodes:
    numeric_categ_features = G.nodes[emp_id].get('features', np.zeros(features_df.shape[1]-1))
    text_features = G.nodes[emp_id].get('text_features', np.zeros(768))
    combined_features = np.concatenate([numeric_categ_features, text_features])
    G.nodes[emp_id]['combined_features'] = combined_features

# Convertir a formato PyTorch Geometric
node_features = torch.tensor([G.nodes[n]['combined_features'] for n in G.nodes], dtype=torch.float)
edge_index = torch.tensor([[e[0], e[1]] for e in G.edges], dtype=torch.long).t()
edge_weight = torch.tensor([G[e[0]][e[1]]['weight'] for e in G.edges], dtype=torch.float)

# Crear objeto Data
data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_weight)

print(f"Dimensiones de node_features: {node_features.shape}")
print(f"Dimensiones de edge_index: {edge_index.shape}")
print(f"Dimensiones de edge_weight: {edge_weight.shape}")

Dimensiones de node_features: torch.Size([60, 806])
Dimensiones de edge_index: torch.Size([2, 1127])
Dimensiones de edge_weight: torch.Size([1127])


<ipython-input-24-1fba999f8978>:11: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  node_features = torch.tensor([G.nodes[n]['combined_features'] for n in G.nodes], dtype=torch.float)


## Resumen
### Grafo construido:

**Nodos**: Emprendimientos con atributos (nombre_emprendimiento, descripcion, redes_sociales, sitio_web, id_municipio_origen, id_alcance, temáticas).

**Aristas**: Basadas en interacciones (likes, temáticas compartidas) y similitudes (temáticas, municipio, alcance), ponderadas por n_likes y seguidores.cantidad.

**Normalización y codificación**:
total_likes y cantidad normalizados.
id_municipio_origen, id_alcance y temáticas codificados con one-hot encoding.
**Procesamiento de texto**: Embeddings de BETO para descripcion, contenido y comentario, combinados por promedio.

**Formato final**: Grafo en formato PyTorch Geometric listo para GraphSAGE.

In [93]:
import torch
import networkx as nx


# Verificar nodos en el grafo
print("Nodos en G:", list(G.nodes)[:10])
print("Número de nodos:", G.number_of_nodes())

# Crear un mapeo de nodos a índices contiguos (0, 1, ..., N-1)
nodes = list(G.nodes)
mapping = {node: idx for idx, node in enumerate(nodes)}

# Verificar que todos los nodos en las aristas estén en el mapeo
edges_to_remove = [e for e in G.edges if e[0] not in mapping or e[1] not in mapping]
if edges_to_remove:
    print("Eliminando aristas con nodos inválidos:", edges_to_remove)
    G.remove_edges_from(edges_to_remove)

# Actualizar edge_index y edge_attr
edge_index_list = [[mapping[e[0]], mapping[e[1]]] for e in G.edges]
edge_weight_list = [G[e[0]][e[1]]['weight'] for e in G.edges]
edge_index = torch.tensor(edge_index_list, dtype=torch.long).t()
edge_weight = torch.tensor(edge_weight_list, dtype=torch.float)

print("Edge index creado con éxito:", edge_index.shape)

Nodos en G: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Número de nodos: 60
Edge index creado con éxito: torch.Size([2, 1127])


In [94]:
# Obtener características de los nodos
node_features = torch.tensor([G.nodes[n]['combined_features'] for n in sorted(G.nodes)], dtype=torch.float)

# Crear objeto Data
data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_weight)

print(f"Dimensiones de node_features: {data.x.shape}")
print(f"Dimensiones de edge_index: {data.edge_index.shape}")
print(f"Dimensiones de edge_weight: {data.edge_attr.shape}")
print(f"Número de nodos: {data.num_nodes}")
print(f"Número de aristas: {data.num_edges}")

Dimensiones de node_features: torch.Size([60, 806])
Dimensiones de edge_index: torch.Size([2, 1127])
Dimensiones de edge_weight: torch.Size([1127])
Número de nodos: 60
Número de aristas: 1127


In [95]:
from torch_geometric.transforms import RandomLinkSplit
from sklearn.preprocessing import StandardScaler

# Normalizar node_features
scaler = StandardScaler()
data.x = torch.tensor(scaler.fit_transform(data.x), dtype=torch.float)

# Verificar varianza de características
print("Media y desv. estándar de node_features:", data.x.mean(), data.x.std())

# Dividir las aristas
transform = RandomLinkSplit(
    num_val=0.15,  # 15% para validación
    num_test=0.15,  # 15% para prueba
    is_undirected=True,  # Grafo no dirigido
    add_negative_train_samples=True,  # Generar aristas negativas
    neg_sampling_ratio=0.5  # Igual número de aristas negativas que positivas
)

try:
    train_data, val_data, test_data = transform(data)
    print(f"Aristas de entrenamiento positivas: {train_data.edge_label_index.shape}")
    print(f"Aristas de validación: {val_data.edge_label_index.shape}")
    print(f"Aristas de prueba: {test_data.edge_label_index.shape}")
except Exception as e:
    print(f"Error al dividir aristas: {e}")
    print("Número de aristas:", data.num_edges)

Media y desv. estándar de node_features: tensor(-7.8881e-11) tensor(1.0000)
Aristas de entrenamiento positivas: torch.Size([2, 1183])
Aristas de validación: torch.Size([2, 253])
Aristas de prueba: torch.Size([2, 253])


In [96]:
from torch_geometric.nn import SAGEConv
import torch.nn.functional as F

class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)
        for layer in [self.conv1, self.conv2]:
            torch.nn.init.xavier_uniform_(layer.lin_l.weight)
            if layer.lin_r is not None:
                torch.nn.init.xavier_uniform_(layer.lin_r.weight)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.6, training=self.training)  # Más dropout
        x = self.conv2(x, edge_index)
        return x

    def predict_link(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=1)

model = GraphSAGE(in_channels=train_data.x.shape[1], hidden_channels=64, out_channels=32)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00005, weight_decay=1e-4)

In [97]:
# Verificar datos
print("Nodos en train_data:", train_data.num_nodes)
print("Aristas en train_data:", train_data.edge_index.shape[1])
print("Aristas etiquetadas (train):", train_data.edge_label_index.shape[1])
print("Balance de etiquetas (train):", train_data.edge_label.unique(return_counts=True))
print("Aristas etiquetadas (val):", val_data.edge_label_index.shape[1])
print("Balance de etiquetas (val):", val_data.edge_label.unique(return_counts=True))

Nodos en train_data: 60
Aristas en train_data: 1578
Aristas etiquetadas (train): 1183
Balance de etiquetas (train): (tensor([0., 1.]), tensor([394, 789]))
Aristas etiquetadas (val): 253
Balance de etiquetas (val): (tensor([0., 1.]), tensor([ 84, 169]))


In [98]:
# Verificar instalaciones
import torch
import torch_geometric
print("PyTorch version:", torch.__version__)
print("PyTorch Geometric version:", torch_geometric.__version__)
try:
    import torch_sparse
    print("torch-sparse instalado correctamente")
except ImportError:
    print("torch-sparse no está instalado")
try:
    import pyg_lib
    print("pyg-lib instalado correctamente")
except ImportError:
    print("pyg-lib no está instalado")

PyTorch version: 2.6.0+cu124
PyTorch Geometric version: 2.5.3
torch-sparse instalado correctamente
pyg-lib instalado correctamente


In [99]:
import torch_sparse
import pyg_lib
from torch_geometric.loader import NeighborLoader
from sklearn.metrics import roc_auc_score, accuracy_score
import copy

# Crear cargador de datos

train_loader = NeighborLoader(
    train_data,
    num_neighbors=[15, 5],
    batch_size=16,
    input_nodes=torch.arange(train_data.num_nodes)
)

"""
train_loader = NeighborLoader(
    train_data,
    num_neighbors=[30, 15, 10],  # Más vecinos y capas
    batch_size=32,  # Ajustar según memoria
    input_nodes=torch.arange(train_data.num_nodes)
)


train_loader = NeighborLoader(
    train_data,
    num_neighbors=[10, 5],  # 10 vecinos en la 1ª capa, 5 en la 2ª
    batch_size=16,
    input_nodes=torch.arange(train_data.num_nodes)
)
"""

# Función de entrenamiento
def train():
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        z = model(batch.x, batch.edge_index)
        n_id = batch.n_id
        node_idx_map = {int(n_id[i]): i for i in range(len(n_id))}
        valid_edges = []
        for src, dst in batch.edge_label_index.t():
            src, dst = int(src), int(dst)
            if src in node_idx_map and dst in node_idx_map:
                valid_edges.append([node_idx_map[src], node_idx_map[dst]])
        if not valid_edges:
            continue
        edge_label_index_subgraph = torch.tensor(valid_edges, dtype=torch.long).t().to(batch.x.device)
        valid_mask = [i for i, (src, dst) in enumerate(batch.edge_label_index.t()) if int(src) in node_idx_map and int(dst) in node_idx_map]
        labels_subgraph = batch.edge_label[valid_mask]
        scores = model.predict_link(z, edge_label_index_subgraph)
        loss = F.binary_cross_entropy_with_logits(scores, labels_subgraph)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

def evaluate(data):
    model.eval()
    with torch.no_grad():
        z = model(data.x, data.edge_index)
        scores = model.predict_link(z, data.edge_label_index).sigmoid()
        labels = data.edge_label
        auc = roc_auc_score(labels.cpu(), scores.cpu())
        preds = (scores > 0.5).float()
        acc = accuracy_score(labels.cpu(), preds.cpu())
    return auc, acc

# Early stopping
best_auc = 0
best_model = None
patience = 30
counter = 0

for epoch in range(1, 601):
    loss = train()
    if epoch % 10 == 0:
        val_auc, val_acc = evaluate(val_data)
        print(f'Época {epoch}, Pérdida: {loss:.4f}, AUC Validación: {val_auc:.4f}, Precisión Validación: {val_acc:.4f}')
        if val_auc > best_auc:
            best_auc = val_auc
            best_model = copy.deepcopy(model.state_dict())
            counter = 0
        else:
            counter += 1
        if counter >= patience:
            print(f"Early stopping en época {epoch}")
            break
    else:
        print(f'Época {epoch}, Pérdida: {loss:.4f}')

# Cargar mejor modelo
if best_model is not None:
    model.load_state_dict(best_model)

Época 1, Pérdida: 13.4786
Época 2, Pérdida: 11.7904
Época 3, Pérdida: 12.1373
Época 4, Pérdida: 11.7522
Época 5, Pérdida: 11.5666
Época 6, Pérdida: 10.2657
Época 7, Pérdida: 10.1513
Época 8, Pérdida: 9.5111
Época 9, Pérdida: 8.5084
Época 10, Pérdida: 9.7641, AUC Validación: 0.5837, Precisión Validación: 0.6719
Época 11, Pérdida: 8.7974
Época 12, Pérdida: 9.3168
Época 13, Pérdida: 8.4952
Época 14, Pérdida: 7.9206
Época 15, Pérdida: 7.2192
Época 16, Pérdida: 8.3089
Época 17, Pérdida: 7.1732
Época 18, Pérdida: 7.1811
Época 19, Pérdida: 6.5550
Época 20, Pérdida: 6.8010, AUC Validación: 0.6377, Precisión Validación: 0.6798
Época 21, Pérdida: 7.0609
Época 22, Pérdida: 6.3630
Época 23, Pérdida: 6.5903
Época 24, Pérdida: 6.6238
Época 25, Pérdida: 6.1752
Época 26, Pérdida: 6.3347
Época 27, Pérdida: 5.7302
Época 28, Pérdida: 6.1229
Época 29, Pérdida: 6.3052
Época 30, Pérdida: 5.6419, AUC Validación: 0.6797, Precisión Validación: 0.7036
Época 31, Pérdida: 5.7247
Época 32, Pérdida: 5.8258
Época 33

In [100]:
from sklearn.metrics import roc_auc_score, accuracy_score

def evaluate(data):
    model.eval()
    with torch.no_grad():
        z = model(data.x, data.edge_index)
        scores = model.predict_link(z, data.edge_label_index).sigmoid()
        labels = data.edge_label
        auc = roc_auc_score(labels.cpu(), scores.cpu())
        preds = (scores > 0.5).float()
        acc = accuracy_score(labels.cpu(), preds.cpu())
    return auc, acc

val_auc, val_acc = evaluate(val_data)
print(f'Validación - AUC: {val_auc:.4f}, Precisión: {val_acc:.4f}')

Validación - AUC: 0.7603, Precisión: 0.7787


In [101]:
# Cargar el mejor modelo guardado
model.load_state_dict(best_model)
model.eval()  # Poner el modelo en modo evaluación

# Obtener los embeddings de todos los nodos
with torch.no_grad():
    z = model(data.x, data.edge_index)  # z contiene los embeddings de todos los nodos

In [102]:
def predict_links(node_idx, top_k=5):
    """
    Predice las top_k conexiones más probables para un nodo dado

    Args:
        node_idx: Índice del nodo (según tu mapeo)
        top_k: Número de recomendaciones a devolver
    """
    # Calcular similitud entre el nodo y todos los demás
    similarities = torch.matmul(z, z[node_idx].unsqueeze(1)).squeeze()

    # Ordenar por similitud descendente
    sorted_sims, indices = torch.sort(similarities, descending=True)

    # Excluir el propio nodo y obtener los top_k
    top_indices = indices[1:top_k+1]  # Excluye el primer elemento (él mismo)
    top_sims = sorted_sims[1:top_k+1]

    # Mapear índices numéricos a IDs originales si es necesario
    # (Asumiendo que tienes un mapeo inverso)
    inverse_mapping = {v: k for k, v in mapping.items()}

    recommendations = []
    for idx, sim in zip(top_indices, top_sims):
        original_id = inverse_mapping[int(idx)]
        recommendations.append((original_id, float(sim)))

    return recommendations

In [103]:
def find_similar_emprendimientos(emprendimiento_id, top_k=5):
    """
    Encuentra emprendimientos similares basado en los embeddings

    Args:
        emprendimiento_id: ID original del emprendimiento en tu base de datos
        top_k: Número de recomendaciones a devolver
    """
    # Convertir ID original a índice del grafo
    node_idx = mapping[emprendimiento_id]

    # Obtener recomendaciones
    recommendations = predict_links(node_idx, top_k)

    # Aquí puedes añadir más lógica para filtrar o procesar los resultados
    return recommendations

In [113]:
# Supongamos que quieres recomendaciones para el emprendimiento con ID 123
emprendimiento_id = 49

try:
    recomendaciones = find_similar_emprendimientos(emprendimiento_id, top_k=10)
    print(f"Recomendaciones para el emprendimiento {emprendimiento_id}:")
    for rec_id, sim_score in recomendaciones:
        print(f"ID: {rec_id}, Similitud: {sim_score:.4f}")

        # Aquí podrías añadir código para obtener más información de tu base de datos
        # Por ejemplo:
        # emprendimiento = obtener_emprendimiento_de_bd(rec_id)
        # print(f"Nombre: {emprendimiento['nombre']}, Descripción: {emprendimiento['descripcion'][:50]}...")

except KeyError:
    print(f"El emprendimiento con ID {emprendimiento_id} no está en el grafo")

Recomendaciones para el emprendimiento 49:
ID: 2, Similitud: 1.9422
ID: 49, Similitud: 1.8283
ID: 54, Similitud: 1.8035
ID: 3, Similitud: 1.7518
ID: 4, Similitud: 1.7227
ID: 5, Similitud: 1.6123
ID: 16, Similitud: 1.4740
ID: 31, Similitud: 1.4528
ID: 17, Similitud: 1.4512
ID: 19, Similitud: 1.3885


In [ ]:
import os
#os.kill(os.getpid(), 9)